In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from transformers import AutoModelForSequenceClassification
import torch.nn.functional as F


/home/yeaz/Desktop/git/sentiment/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
a = pd.read_csv('datasets/spotify/dataset.csv')
b = pd.read_csv('datasets/spotify/MusicInfo.csv')

In [3]:
a

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,...,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
113996,113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,...,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
113997,113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,...,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
113998,113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,...,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [4]:
a.columns

Index(['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name',
       'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'track_genre'],
      dtype='object')

In [5]:
b

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50678,TRQYCFV128F9322F50,Ryusei Rocket,アンティック-珈琲店-,https://p.scdn.co/mp3-preview/d2668a5a3e0b1fda...,0tt1RdeJX1RyuU4hMEZ19T,NaN,NaN,2008,273440,0.438,...,6,-3.062,0,0.1650,0.003120,0.000000,0.1300,0.421,166.956,4
50679,TRHQCSH128F42724B7,Colors Of The Wind,ACIDMAN,https://p.scdn.co/mp3-preview/8e22a7052ef3ecf7...,3wkdfXGf5JYErW4b35zP2h,"rock, alternative_rock, japanese, cover",NaN,2004,275133,0.351,...,0,-6.811,1,0.1200,0.000940,0.000049,0.1920,0.450,200.350,4
50680,TRZRODK128F92D68D7,The Revelation,coldrain,https://p.scdn.co/mp3-preview/4b51a813f67e3853...,1gXMORZRGA40PE9rDE9cja,"metal, metalcore, post_hardcore",NaN,2014,254826,0.434,...,10,-3.092,0,0.2680,0.000108,0.001410,0.1630,0.282,158.025,4
50681,TRGLMEM128F9322F63,Koi no Dependence,アンティック-珈琲店-,https://p.scdn.co/mp3-preview/5a61e031df174666...,1dxMDGvIYHFYgRvmw1uMHG,NaN,NaN,2008,243293,0.513,...,4,-3.914,0,0.0530,0.000715,0.001350,0.0571,0.618,109.923,4


In [6]:
b.columns

Index(['track_id', 'name', 'artist', 'spotify_preview_url', 'spotify_id',
       'tags', 'genre', 'year', 'duration_ms', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature'],
      dtype='object')

In [7]:
a = a[['track_id', 'artists', 'track_name', 'duration_ms', 'danceability', 'energy','key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
a.rename(columns={'track_name': 'name'}, inplace=True)

/tmp/ipykernel_5117/513043980.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a.rename(columns={'track_name': 'name'}, inplace=True)


In [8]:
b = b[['name', 'artist', 'spotify_id', 'duration_ms', 'danceability', 'energy', 'key','loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness','liveness', 'valence', 'tempo', 'time_signature']]
b.rename(columns={'spotify_id': 'track_id', 'artist': 'artists'}, inplace=True)

/tmp/ipykernel_5117/3633656779.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b.rename(columns={'spotify_id': 'track_id', 'artist': 'artists'}, inplace=True)


In [9]:
common_columns = list(set(a.columns) & set(b.columns))
common_columns

['tempo',
 'energy',
 'mode',
 'valence',
 'speechiness',
 'danceability',
 'artists',
 'name',
 'time_signature',
 'track_id',
 'duration_ms',
 'key',
 'loudness',
 'liveness',
 'acousticness',
 'instrumentalness']

In [10]:
non_matching_columns = list(set(a.columns).symmetric_difference(b.columns))
non_matching_columns

[]

In [11]:
data = pd.concat([a, b], ignore_index=True, sort=False)
data

,track_id,artists,name,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,230666,0.676,0.4610,1,-6.746,0,0.1430,0.032200,0.000001,0.3580,0.715,87.917,4
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost - Acoustic,149610,0.420,0.1660,1,-17.235,1,0.0763,0.924000,0.000006,0.1010,0.267,77.489,4
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,210826,0.438,0.3590,0,-9.734,1,0.0557,0.210000,0.000000,0.1170,0.120,76.332,4
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Can't Help Falling In Love,201933,0.266,0.0596,0,-18.515,1,0.0363,0.905000,0.000071,0.1320,0.143,181.740,3
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,198853,0.618,0.4430,2,-9.681,1,0.0526,0.469000,0.000000,0.0829,0.167,119.949,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164678,0tt1RdeJX1RyuU4hMEZ19T,アンティック-珈琲店-,Ryusei Rocket,273440,0.438,0.9330,6,-3.062,0,0.1650,0.003120,0.000000,0.1300,0.421,166.956,4
164679,3wkdfXGf5JYErW4b35zP2h,ACIDMAN,Colors Of The Wind,275133,0.351,0.6930,0,-6.811,1,0.1200,0.000940,0.000049,0.1920,0.450,200.350,4
164680,1gXMORZRGA40PE9rDE9cja,coldrain,The Revelation,254826,0.434,0.9750,10,-3.092,0,0.2680,0.000108,0.001410,0.1630,0.282,158.025,4
164681,1dxMDGvIYHFYgRvmw1uMHG,アンティック-珈琲店-,Koi no Dependence,243293,0.513,0.9020,4,-3.914,0,0.0530,0.000715,0.001350,0.0571,0.618,109.923,4


In [12]:
data.columns

Index(['track_id', 'artists', 'name', 'duration_ms', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'],
      dtype='object')

In [13]:
ax = pd.read_csv('mainData.csv')
ax['emotion'] = ax['emotion'].apply(eval)
ax

,text,emotion
0,[RU] Эта игра причинила мне боль.,[sadness]
1,"[RU] Ты правильно поступаешь, если тебе все ра...",[neutral]
2,"[RU] Чувак, я обожаю Reddit.",[love]
3,"[RU] [ИМЯ] не было рядом с ними, он был рядом ...",[neutral]
4,"[RU] Верно? Учитывая, что это такой важный док...",[gratitude]
...,...,...
479720,"[KZ] ""Сен онымен на хабаре болсаң, менің өтіні...","[desire, caring, nervousness]"
479721,"[KZ] ""Оның ""танкасы бар"" болса да, заң алдында...","[approval, determination, anger]"
479722,"[KZ] ""Ол шындықты айтып тұр ма, или өтірік пе,...","[curiosity, nervousness, confusion]"
479723,"[KZ] Университетте оқып жүргенде, біз общягада...","[nostalgia, caring, joy]"


In [14]:
unique = list(set(emotion for sublist in ax['emotion'] for emotion in sublist))
print(unique)

['regret', 'shame', 'confidence', 'urgency', 'relief', 'surprise', 'envy', 'shock', 'disgust', 'optimism', 'panic', 'embarrassment', 'satisfaction', 'frustration', 'realization', 'gratitude', 'disappointment', 'creativity', 'hope', 'approv', 'love', 'tradition', 'annoyance', 'grief', 'discomfort', 'deceit', 'contentment', 'fear', 'admiration', 'concern', 'anger', 'neutral', 'confusion', 'sadness', 'anticipation', 'curiosity', 'caring', 'joy', 'amusement', 'nervousness', 'disapproval', 'remorse', 'suspicion', 'nostalgia', 'disbelief', 'caution', 'reassurance', 'trust', 'passion', 'approval', 'anxiety', 'desire', 'pride', 'excitement', 'determination']


In [15]:
data

,track_id,artists,name,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,230666,0.676,0.4610,1,-6.746,0,0.1430,0.032200,0.000001,0.3580,0.715,87.917,4
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost - Acoustic,149610,0.420,0.1660,1,-17.235,1,0.0763,0.924000,0.000006,0.1010,0.267,77.489,4
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,210826,0.438,0.3590,0,-9.734,1,0.0557,0.210000,0.000000,0.1170,0.120,76.332,4
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Can't Help Falling In Love,201933,0.266,0.0596,0,-18.515,1,0.0363,0.905000,0.000071,0.1320,0.143,181.740,3
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,198853,0.618,0.4430,2,-9.681,1,0.0526,0.469000,0.000000,0.0829,0.167,119.949,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164678,0tt1RdeJX1RyuU4hMEZ19T,アンティック-珈琲店-,Ryusei Rocket,273440,0.438,0.9330,6,-3.062,0,0.1650,0.003120,0.000000,0.1300,0.421,166.956,4
164679,3wkdfXGf5JYErW4b35zP2h,ACIDMAN,Colors Of The Wind,275133,0.351,0.6930,0,-6.811,1,0.1200,0.000940,0.000049,0.1920,0.450,200.350,4
164680,1gXMORZRGA40PE9rDE9cja,coldrain,The Revelation,254826,0.434,0.9750,10,-3.092,0,0.2680,0.000108,0.001410,0.1630,0.282,158.025,4
164681,1dxMDGvIYHFYgRvmw1uMHG,アンティック-珈琲店-,Koi no Dependence,243293,0.513,0.9020,4,-3.914,0,0.0530,0.000715,0.001350,0.0571,0.618,109.923,4


In [16]:
data.shape

(164683, 16)

In [17]:
data.drop_duplicates(subset=['artists', 'name'])
data.shape

(164683, 16)

In [18]:
import requests
import base64
import time
from tqdm import tqdm

client_id = '9f201534908947089c9e9af85196a1f0'
client_secret = 'fb08f6071324494cb8f98adc6412cc06'

def get_spotify_token(client_id, client_secret):
    auth_str = f"{client_id}:{client_secret}"
    b64_auth_str = base64.b64encode(auth_str.encode()).decode()
    token_url = 'https://accounts.spotify.com/api/token'
    response = requests.post(
        token_url,
        data={'grant_type': 'client_credentials'},
        headers={'Authorization': f'Basic {b64_auth_str}'}
    )
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        raise Exception(f"Failed to get token: {response.status_code}, {response.text}")


In [19]:
def get_track_popularity(track_id, token, retries=3):
    url = f'https://api.spotify.com/v1/tracks/{track_id}'
    headers = {'Authorization': f'Bearer {token}'}

    for attempt in range(retries):
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            return response.json().get('popularity', None)

        elif response.status_code == 429:
            retry_after = int(response.headers.get('Retry-After', 5))
            print(f"Rate limited. Sleeping for {retry_after} seconds...")
            time.sleep(retry_after)

        else:
            print(f"Error {response.status_code} on track {track_id}")
            time.sleep(2)
    return None

In [20]:
CHECKPOINT_FILE = 'MusicsUpdated.csv'

data = pd.read_csv(CHECKPOINT_FILE)

if 'popularity' not in data.columns:
    data['popularity'] = pd.NA

token = get_spotify_token(client_id, client_secret)

for idx, row in tqdm(data[data['popularity'].isna()].iterrows(), total=data['popularity'].isna().sum()):
    track_id = row['track_id']
    try:
        pop = get_track_popularity(track_id, token)
        data.at[idx, 'popularity'] = pop
    except Exception as e:
        print(f"Stopping due to error: {e}")
        break

    if idx % 100 == 0:
        time.sleep(1)

    if idx % 800 == 0:
        data.to_csv(CHECKPOINT_FILE, index=False)
        print("💾 Progress saved.")

data.to_csv(CHECKPOINT_FILE, index=False)

Exception: Failed to get token: 400, {"error":"invalid_client","error_description":"Failed to get client"}

In [21]:
aa = pd.read_csv('MusicsUpdated.csv')
aa

,track_id,artists,name,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,popularity
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,230666,0.676,0.4610,1,-6.746,0,0.1430,0.032200,0.000001,0.3580,0.715,87.917,4,65.0
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost - Acoustic,149610,0.420,0.1660,1,-17.235,1,0.0763,0.924000,0.000006,0.1010,0.267,77.489,4,45.0
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,210826,0.438,0.3590,0,-9.734,1,0.0557,0.210000,0.000000,0.1170,0.120,76.332,4,52.0
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Can't Help Falling In Love,201933,0.266,0.0596,0,-18.515,1,0.0363,0.905000,0.000071,0.1320,0.143,181.740,3,70.0
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,198853,0.618,0.4430,2,-9.681,1,0.0526,0.469000,0.000000,0.0829,0.167,119.949,4,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164678,0tt1RdeJX1RyuU4hMEZ19T,アンティック-珈琲店-,Ryusei Rocket,273440,0.438,0.9330,6,-3.062,0,0.1650,0.003120,0.000000,0.1300,0.421,166.956,4,0.0
164679,3wkdfXGf5JYErW4b35zP2h,ACIDMAN,Colors Of The Wind,275133,0.351,0.6930,0,-6.811,1,0.1200,0.000940,0.000049,0.1920,0.450,200.350,4,12.0
164680,1gXMORZRGA40PE9rDE9cja,coldrain,The Revelation,254826,0.434,0.9750,10,-3.092,0,0.2680,0.000108,0.001410,0.1630,0.282,158.025,4,0.0
164681,1dxMDGvIYHFYgRvmw1uMHG,アンティック-珈琲店-,Koi no Dependence,243293,0.513,0.9020,4,-3.914,0,0.0530,0.000715,0.001350,0.0571,0.618,109.923,4,0.0


In [ ]:
data.to_csv('Musics.csv', index=False)

In [22]:
unique_emotions = ['suspicion', 'disappointment', 'hope', 'curiosity', 'amusement', 
                   'realization', 'shame', 'reassurance', 'envy', 'disgust', 'fear', 
                   'concern', 'discomfort', 'frustration', 'creativity', 'contentment', 
                   'tradition', 'urgency', 'panic', 'pride', 'nervousness', 'relief', 
                   'nostalgia', 'gratitude', 'shock', 'joy', 'annoyance', 'excitement', 
                   'passion', 'love', 'anger', 'sadness', 'anticipation', 'neutral', 
                   'disapproval', 'admiration', 'caring', 'approval', 'determination', 
                   'satisfaction', 'caution', 'optimism', 'approv', 'surprise', 
                   'anxiety', 'trust', 'confidence', 'regret', 'remorse', 'confusion', 
                   'grief', 'embarrassment', 'deceit', 'desire', 'disbelief']

features_for_emotion_rules = [
    'danceability', 'energy', 'valence', 'tempo', 'loudness', 'mode',
    'speechiness', 'acousticness', 'instrumentalness', 'liveness'
]

In [23]:
len(unique_emotions)

55

In [24]:
columns = ['Emotion']
for feature in features_for_emotion_rules:
    columns.append(f"{feature}_Min")
    columns.append(f"{feature}_Max")

emotion_rules_list = []    

In [25]:
columns

['Emotion',
 'danceability_Min',
 'danceability_Max',
 'energy_Min',
 'energy_Max',
 'valence_Min',
 'valence_Max',
 'tempo_Min',
 'tempo_Max',
 'loudness_Min',
 'loudness_Max',
 'mode_Min',
 'mode_Max',
 'speechiness_Min',
 'speechiness_Max',
 'acousticness_Min',
 'acousticness_Max',
 'instrumentalness_Min',
 'instrumentalness_Max',
 'liveness_Min',
 'liveness_Max']

In [26]:
def create_default_rule(emotion_name):
    rule = {'Emotion': emotion_name}

    rule.update({
        'danceability_Min': 0.0, 'danceability_Max': 1.0,
        'energy_Min': 0.0, 'energy_Max': 1.0,
        'valence_Min': 0.0, 'valence_Max': 1.0,
        'tempo_Min': 50, 'tempo_Max': 200,
        'loudness_Min': -60, 'loudness_Max': 0,
        'mode_Min': 0, 'mode_Max': 1,
        'speechiness_Min': 0.0, 'speechiness_Max': 0.33,
        'acousticness_Min': 0.0, 'acousticness_Max': 1.0,
        'instrumentalness_Min': 0.0, 'instrumentalness_Max': 1.0,
        'liveness_Min': 0.0, 'liveness_Max': 1.0,
    })
    return rule

In [27]:
for emotion in unique_emotions:
    rule = create_default_rule(emotion)

    # --- POSITIVE EMOTIONS ---

    # High Arousal, High Valence (Ecstatic, Joyful, Excited)
    if emotion == 'joy':
        rule.update({
            'danceability_Min': 0.65, 'danceability_Max': 0.9,
            'energy_Min': 0.75, 'energy_Max': 0.95,
            'valence_Min': 0.75, 'valence_Max': 1.0,
            'tempo_Min': 120, 'tempo_Max': 150,
            'loudness_Min': -8, 'loudness_Max': -2,
            'mode_Min': 1, 'mode_Max': 1, # Major
            'speechiness_Min': 0.0, 'speechiness_Max': 0.2,
            'acousticness_Min': 0.0, 'acousticness_Max': 0.4,
            'instrumentalness_Min': 0.0, 'instrumentalness_Max': 0.05
        })
    elif emotion == 'excitement':
        rule.update({
            'danceability_Min': 0.6, 'danceability_Max': 0.85,
            'energy_Min': 0.8, 'energy_Max': 1.0,
            'valence_Min': 0.65, 'valence_Max': 1.0,
            'tempo_Min': 130, 'tempo_Max': 170,
            'loudness_Min': -7, 'loudness_Max': 0,
            'mode_Min': 0, 'mode_Max': 1, # Can be intense minor or major
            'speechiness_Min': 0.05, 'speechiness_Max': 0.3,
            'instrumentalness_Min': 0.0, 'instrumentalness_Max': 0.1
        })
    elif emotion == 'amusement':
        rule.update({
            'danceability_Min': 0.65, 'danceability_Max': 0.9,
            'energy_Min': 0.6, 'energy_Max': 0.85,
            'valence_Min': 0.7, 'valence_Max': 1.0,
            'tempo_Min': 110, 'tempo_Max': 140,
            'mode_Min': 1, 'mode_Max': 1, # Major
            'speechiness_Min': 0.05, 'speechiness_Max': 0.4, # Can have more playful lyrics
            'loudness_Min': -10, 'loudness_Max': -3,
        })
    elif emotion == 'optimism':
        rule.update({
            'danceability_Min': 0.5, 'danceability_Max': 0.8,
            'energy_Min': 0.6, 'energy_Max': 0.85,
            'valence_Min': 0.7, 'valence_Max': 1.0,
            'tempo_Min': 110, 'tempo_Max': 140,
            'mode_Min': 1, 'mode_Max': 1, # Major
            'loudness_Min': -11, 'loudness_Max': -4,
            'acousticness_Min': 0.0, 'acousticness_Max': 0.5
        })

    # Mid/High Arousal, High Valence (Passionate, Loving, Proud)
    elif emotion == 'passion':
        rule.update({
            'danceability_Min': 0.4, 'danceability_Max': 0.75,
            'energy_Min': 0.65, 'energy_Max': 0.9,
            'valence_Min': 0.55, 'valence_Max': 0.9,
            'tempo_Min': 100, 'tempo_Max': 150,
            'loudness_Min': -12, 'loudness_Max': -3,
            'mode_Min': 0, 'mode_Max': 1,
            'speechiness_Min': 0.03, 'speechiness_Max': 0.35
        })
    elif emotion == 'love':
        rule.update({
            'danceability_Min': 0.35, 'danceability_Max': 0.7,
            'energy_Min': 0.45, 'energy_Max': 0.8,
            'valence_Min': 0.65, 'valence_Max': 1.0,
            'tempo_Min': 80, 'tempo_Max': 130,
            'loudness_Min': -15, 'loudness_Max': -5,
            'mode_Min': 0, 'mode_Max': 1, # Often major, but can be soulful minor
            'acousticness_Min': 0.1, 'acousticness_Max': 0.7,
            'speechiness_Min': 0.02, 'speechiness_Max': 0.25
        })
    elif emotion == 'desire':
        rule.update({
            'danceability_Min': 0.5, 'danceability_Max': 0.8,
            'energy_Min': 0.6, 'energy_Max': 0.85,
            'valence_Min': 0.5, 'valence_Max': 0.85,
            'tempo_Min': 90, 'tempo_Max': 140,
            'loudness_Min': -13, 'loudness_Max': -4,
            'mode_Min': 0, 'mode_Max': 1,
            'speechiness_Min': 0.04, 'speechiness_Max': 0.3
        })
    elif emotion == 'admiration':
        rule.update({
            'danceability_Min': 0.3, 'danceability_Max': 0.65,
            'energy_Min': 0.4, 'energy_Max': 0.75,
            'valence_Min': 0.6, 'valence_Max': 0.9,
            'tempo_Min': 80, 'tempo_Max': 130,
            'mode_Min': 1, 'mode_Max': 1, # Often major
            'loudness_Min': -16, 'loudness_Max': -6,
            'acousticness_Min': 0.2, 'acousticness_Max': 0.8
        })
    elif emotion == 'pride':
        rule.update({
            'danceability_Min': 0.3, 'danceability_Max': 0.7,
            'energy_Min': 0.55, 'energy_Max': 0.85,
            'valence_Min': 0.6, 'valence_Max': 0.9,
            'tempo_Min': 90, 'tempo_Max': 140, # Can be anthemic
            'mode_Min': 1, 'mode_Max': 1, # Usually major
            'loudness_Min': -12, 'loudness_Max': -4,
        })
    elif emotion == 'confidence' or emotion == 'determination':
        rule.update({
            'danceability_Min': 0.4, 'danceability_Max': 0.75,
            'energy_Min': 0.6, 'energy_Max': 0.9,
            'valence_Min': 0.5, 'valence_Max': 0.85,
            'tempo_Min': 100, 'tempo_Max': 150,
            'mode_Min': 0, 'mode_Max': 1, # Strong major or driving minor
            'loudness_Min': -10, 'loudness_Max': -3,
        })

    # Low/Mid Arousal, High Valence (Calm, Content, Hopeful)
    elif emotion == 'contentment' or emotion == 'satisfaction':
        rule.update({
            'danceability_Min': 0.2, 'danceability_Max': 0.6,
            'energy_Min': 0.2, 'energy_Max': 0.55,
            'valence_Min': 0.7, 'valence_Max': 1.0,
            'tempo_Min': 70, 'tempo_Max': 115,
            'loudness_Min': -20, 'loudness_Max': -9,
            'mode_Min': 1, 'mode_Max': 1, # Major
            'acousticness_Min': 0.5, 'acousticness_Max': 1.0,
            'instrumentalness_Min': 0.1, 'instrumentalness_Max': 0.8,
            'speechiness_Min': 0.0, 'speechiness_Max': 0.15
        })
    elif emotion == 'relief':
        rule.update({
            'danceability_Min': 0.15, 'danceability_Max': 0.5,
            'energy_Min': 0.1, 'energy_Max': 0.45,
            'valence_Min': 0.6, 'valence_Max': 0.9,
            'tempo_Min': 60, 'tempo_Max': 110,
            'loudness_Min': -24, 'loudness_Max': -10,
            'mode_Min': 1, 'mode_Max': 1,
            'acousticness_Min': 0.6, 'acousticness_Max': 1.0,
        })
    elif emotion == 'caring':
        rule.update({
            'danceability_Min': 0.2, 'danceability_Max': 0.55,
            'energy_Min': 0.2, 'energy_Max': 0.5,
            'valence_Min': 0.6, 'valence_Max': 0.9,
            'tempo_Min': 65, 'tempo_Max': 110,
            'loudness_Min': -20, 'loudness_Max': -9,
            'mode_Min': 1, 'mode_Max': 1,
            'acousticness_Min': 0.5, 'acousticness_Max': 1.0,
            'speechiness_Min': 0.0, 'speechiness_Max': 0.2
        })
    elif emotion == 'gratitude' or emotion == 'reassurance' or emotion == 'trust':
        rule.update({
            'danceability_Min': 0.2, 'danceability_Max': 0.6,
            'energy_Min': 0.25, 'energy_Max': 0.6,
            'valence_Min': 0.65, 'valence_Max': 0.95,
            'tempo_Min': 70, 'tempo_Max': 120,
            'mode_Min': 1, 'mode_Max': 1,
            'loudness_Min': -18, 'loudness_Max': -7,
            'acousticness_Min': 0.4, 'acousticness_Max': 1.0
        })
    elif emotion == 'hope':
        rule.update({
            'danceability_Min': 0.25, 'danceability_Max': 0.65,
            'energy_Min': 0.3, 'energy_Max': 0.65,
            'valence_Min': 0.55, 'valence_Max': 0.9, # Can be slightly less overtly positive than pure joy
            'tempo_Min': 80, 'tempo_Max': 125,
            'mode_Min': 0, 'mode_Max': 1, # Can be uplifting minor too
            'loudness_Min': -17, 'loudness_Max': -6,
            'acousticness_Min': 0.3, 'acousticness_Max': 0.9
        })
    elif emotion == 'approval' or emotion == 'approv':
        rule.update({
            'danceability_Min': 0.4, 'danceability_Max': 0.75,
            'energy_Min': 0.45, 'energy_Max': 0.75,
            'valence_Min': 0.6, 'valence_Max': 0.9,
            'tempo_Min': 90, 'tempo_Max': 135,
            'mode_Min': 1, 'mode_Max': 1, # Usually positive/major
            'loudness_Min': -14, 'loudness_Max': -5,
        })

    # --- NEGATIVE EMOTIONS ---

    # High Arousal, Low Valence (Anger, Frustration, Panic)
    elif emotion == 'anger':
        rule.update({
            'danceability_Min': 0.3, 'danceability_Max': 0.65, # Can be driving, not necessarily "dancey"
            'energy_Min': 0.75, 'energy_Max': 1.0,
            'valence_Min': 0.0, 'valence_Max': 0.25,
            'tempo_Min': 125, 'tempo_Max': 180,
            'loudness_Min': -9, 'loudness_Max': 0,
            'mode_Min': 0, 'mode_Max': 0, # Minor
            'speechiness_Min': 0.05, 'speechiness_Max': 0.5, # Can have aggressive vocals
            'instrumentalness_Min': 0.0, 'instrumentalness_Max': 0.4,
            'acousticness_Min': 0.0, 'acousticness_Max': 0.2
        })
    elif emotion == 'frustration':
        rule.update({
            'danceability_Min': 0.25, 'danceability_Max': 0.6,
            'energy_Min': 0.65, 'energy_Max': 0.9,
            'valence_Min': 0.05, 'valence_Max': 0.3,
            'tempo_Min': 110, 'tempo_Max': 170,
            'mode_Min': 0, 'mode_Max': 0,
            'loudness_Min': -11, 'loudness_Max': -2,
        })
    elif emotion == 'annoyance':
        rule.update({
            'danceability_Min': 0.3, 'danceability_Max': 0.65,
            'energy_Min': 0.55, 'energy_Max': 0.8,
            'valence_Min': 0.1, 'valence_Max': 0.4,
            'tempo_Min': 100, 'tempo_Max': 160,
            'mode_Min': 0, 'mode_Max': 1, # Can be agitated major too
            'loudness_Min': -13, 'loudness_Max': -5,
            'speechiness_Min': 0.04, 'speechiness_Max': 0.4
        })
    elif emotion == 'panic' or emotion == 'urgency':
        rule.update({
            'danceability_Min': 0.2, 'danceability_Max': 0.55,
            'energy_Min': 0.8, 'energy_Max': 1.0,
            'valence_Min': 0.0, 'valence_Max': 0.2,
            'tempo_Min': 140, 'tempo_Max': 200,
            'loudness_Min': -8, 'loudness_Max': 0,
            'mode_Min': 0, 'mode_Max': 0,
            'speechiness_Min': 0.05, 'speechiness_Max': 0.4,
            'instrumentalness_Min': 0.1, 'instrumentalness_Max': 0.7 # Can be chaotic instrumental
        })

    # Mid/High Arousal, Low Valence (Fear, Anxiety, Disgust)
    elif emotion == 'fear' or emotion == 'anxiety':
        rule.update({
            'danceability_Min': 0.15, 'danceability_Max': 0.45,
            'energy_Min': 0.4, 'energy_Max': 0.75, # Can build
            'valence_Min': 0.0, 'valence_Max': 0.25,
            'tempo_Min': 70, 'tempo_Max': 150, # Can be slow and building or erratically fast
            'loudness_Min': -19, 'loudness_Max': -7,
            'mode_Min': 0, 'mode_Max': 0, # Minor
            'acousticness_Min': 0.1, 'acousticness_Max': 0.8, # Can be sparse or dissonant
            'instrumentalness_Min': 0.2, 'instrumentalness_Max': 0.9,
        })
    elif emotion == 'nervousness':
        rule.update({
            'danceability_Min': 0.2, 'danceability_Max': 0.5,
            'energy_Min': 0.35, 'energy_Max': 0.7,
            'valence_Min': 0.1, 'valence_Max': 0.35,
            'tempo_Min': 90, 'tempo_Max': 160, # Often agitated
            'mode_Min': 0, 'mode_Max': 0,
            'loudness_Min': -17, 'loudness_Max': -8,
        })
    elif emotion == 'discomfort':
        rule.update({
            'danceability_Min': 0.1, 'danceability_Max': 0.4,
            'energy_Min': 0.2, 'energy_Max': 0.5,
            'valence_Min': 0.1, 'valence_Max': 0.3,
            'tempo_Min': 60, 'tempo_Max': 110,
            'mode_Min': 0, 'mode_Max': 0,
            'loudness_Min': -20, 'loudness_Max': -9,
            'acousticness_Min': 0.3, 'acousticness_Max': 0.9
        })
    elif emotion == 'suspicion' or emotion == 'caution':
        rule.update({
            'danceability_Min': 0.1, 'danceability_Max': 0.4,
            'energy_Min': 0.15, 'energy_Max': 0.45,
            'valence_Min': 0.1, 'valence_Max': 0.35,
            'tempo_Min': 60, 'tempo_Max': 110,
            'mode_Min': 0, 'mode_Max': 0,
            'loudness_Min': -22, 'loudness_Max': -10,
            'instrumentalness_Min': 0.4, 'instrumentalness_Max': 1.0, # Often more instrumental, suspenseful
            'acousticness_Min': 0.2, 'acousticness_Max': 0.8
        })
    elif emotion == 'disgust':
        rule.update({
            'danceability_Min': 0.1, 'danceability_Max': 0.4,
            'energy_Min': 0.25, 'energy_Max': 0.6,
            'valence_Min': 0.0, 'valence_Max': 0.15, # Very low
            'tempo_Min': 70, 'tempo_Max': 130,
            'mode_Min': 0, 'mode_Max': 0,
            'loudness_Min': -19, 'loudness_Max': -8,
            'speechiness_Min': 0.0, 'speechiness_Max': 0.3,
        })

    # Low Arousal, Low Valence (Sadness, Grief, Disappointment)
    elif emotion == 'sadness':
        rule.update({
            'danceability_Min': 0.1, 'danceability_Max': 0.35,
            'energy_Min': 0.1, 'energy_Max': 0.35,
            'valence_Min': 0.0, 'valence_Max': 0.25,
            'tempo_Min': 55, 'tempo_Max': 95,
            'loudness_Min': -23, 'loudness_Max': -11,
            'mode_Min': 0, 'mode_Max': 0, # Minor
            'acousticness_Min': 0.6, 'acousticness_Max': 1.0,
            'speechiness_Min': 0.0, 'speechiness_Max': 0.15,
            'instrumentalness_Min': 0.0, 'instrumentalness_Max': 0.7
        })
    elif emotion == 'grief':
        rule.update({
            'danceability_Min': 0.05, 'danceability_Max': 0.3,
            'energy_Min': 0.0, 'energy_Max': 0.25,
            'valence_Min': 0.0, 'valence_Max': 0.15,
            'tempo_Min': 50, 'tempo_Max': 85,
            'loudness_Min': -28, 'loudness_Max': -14,
            'mode_Min': 0, 'mode_Max': 0,
            'acousticness_Min': 0.75, 'acousticness_Max': 1.0,
            'instrumentalness_Min': 0.3, 'instrumentalness_Max': 0.95,
            'speechiness_Min': 0.0, 'speechiness_Max': 0.1
        })
    elif emotion == 'disappointment':
        rule.update({
            'danceability_Min': 0.1, 'danceability_Max': 0.4,
            'energy_Min': 0.15, 'energy_Max': 0.4,
            'valence_Min': 0.05, 'valence_Max': 0.3,
            'tempo_Min': 60, 'tempo_Max': 100,
            'mode_Min': 0, 'mode_Max': 0,
            'loudness_Min': -22, 'loudness_Max': -10,
            'acousticness_Min': 0.4, 'acousticness_Max': 0.95
        })
    elif emotion == 'regret' or emotion == 'remorse':
        rule.update({
            'danceability_Min': 0.1, 'danceability_Max': 0.4,
            'energy_Min': 0.1, 'energy_Max': 0.35,
            'valence_Min': 0.05, 'valence_Max': 0.35, # Can have a touch more complexity
            'tempo_Min': 55, 'tempo_Max': 90,
            'mode_Min': 0, 'mode_Max': 0,
            'loudness_Min': -24, 'loudness_Max': -11,
            'acousticness_Min': 0.6, 'acousticness_Max': 1.0
        })
    elif emotion == 'shame' or emotion == 'embarrassment':
        rule.update({
            'danceability_Min': 0.15, 'danceability_Max': 0.45,
            'energy_Min': 0.2, 'energy_Max': 0.45,
            'valence_Min': 0.1, 'valence_Max': 0.3,
            'tempo_Min': 65, 'tempo_Max': 105,
            'mode_Min': 0, 'mode_Max': 0,
            'loudness_Min': -21, 'loudness_Max': -10,
            'speechiness_Min': 0.0, 'speechiness_Max': 0.25
        })

    # --- COGNITIVE/ABSTRACT/COMPLEX EMOTIONS ---
    elif emotion == 'curiosity':
        rule.update({
            'danceability_Min': 0.3, 'danceability_Max': 0.7,
            'energy_Min': 0.4, 'energy_Max': 0.75,
            'valence_Min': 0.3, 'valence_Max': 0.7,
            'tempo_Min': 80, 'tempo_Max': 130,
            'instrumentalness_Min': 0.4, 'instrumentalness_Max': 1.0,
            'acousticness_Min': 0.1, 'acousticness_Max': 0.7,
            'time_signature_Min': 3, 'time_signature_Max': 5, # Can be slightly unconventional
            'loudness_Min': -18, 'loudness_Max': -7,
        })
    elif emotion == 'creativity':
        rule.update({
            'danceability_Min': 0.2, 'danceability_Max': 0.75,
            'energy_Min': 0.3, 'energy_Max': 0.8,
            'valence_Min': 0.3, 'valence_Max': 0.85, # Can be quite positive
            'tempo_Min': 70, 'tempo_Max': 150,
            'instrumentalness_Min': 0.2, 'instrumentalness_Max': 0.9,
            'acousticness_Min': 0.05, 'acousticness_Max': 0.75,
            'time_signature_Min': 3, 'time_signature_Max': 7, # More room for complexity
            'loudness_Min': -16, 'loudness_Max': -5,
        })
    elif emotion == 'realization':
        rule.update({
            'danceability_Min': 0.2, 'danceability_Max': 0.6,
            'energy_Min': 0.25, 'energy_Max': 0.65,
            'valence_Min': 0.25, 'valence_Max': 0.7, # Can be neutral to slightly positive/negative
            'tempo_Min': 60, 'tempo_Max': 120, # Often slower, thoughtful
            'instrumentalness_Min': 0.3, 'instrumentalness_Max': 0.9,
            'acousticness_Min': 0.3, 'acousticness_Max': 0.9,
            'loudness_Min': -20, 'loudness_Max': -8,
        })
    elif emotion == 'anticipation':
        rule.update({
            'danceability_Min': 0.3, 'danceability_Max': 0.7,
            'energy_Min': 0.5, 'energy_Max': 0.8, # Building energy
            'valence_Min': 0.4, 'valence_Max': 0.8, # Generally positive leaning
            'tempo_Min': 90, 'tempo_Max': 140, # Can be building
            'instrumentalness_Min': 0.1, 'instrumentalness_Max': 0.7,
            'loudness_Min': -15, 'loudness_Max': -5,
        })
    elif emotion == 'surprise' or emotion == 'shock':
        rule.update({ # Sudden changes, dynamic shifts
            'danceability_Min': 0.2, 'danceability_Max': 0.65,
            'energy_Min': 0.55, 'energy_Max': 0.95,
            'valence_Min': 0.1, 'valence_Max': 0.7, # Can be good or bad surprise
            'tempo_Min': 70, 'tempo_Max': 170, # Can be abrupt
            'loudness_Min': -14, 'loudness_Max': -1, # Can have sudden loudness
            'instrumentalness_Min': 0.0, 'instrumentalness_Max': 0.6,
            'time_signature_Min': 3, 'time_signature_Max': 5,
        })
    elif emotion == 'disbelief':
        rule.update({
            'danceability_Min': 0.2, 'danceability_Max': 0.55,
            'energy_Min': 0.3, 'energy_Max': 0.65,
            'valence_Min': 0.15, 'valence_Max': 0.45, # Leaning negative or confused
            'tempo_Min': 60, 'tempo_Max': 110,
            'instrumentalness_Min': 0.2, 'instrumentalness_Max': 0.8,
            'loudness_Min': -19, 'loudness_Max': -8,
        })
    elif emotion == 'confusion':
        rule.update({
            'danceability_Min': 0.15, 'danceability_Max': 0.5,
            'energy_Min': 0.25, 'energy_Max': 0.6,
            'valence_Min': 0.2, 'valence_Max': 0.5,
            'tempo_Min': 65, 'tempo_Max': 120, # Can be disjointed
            'instrumentalness_Min': 0.1, 'instrumentalness_Max': 0.9, # Can be more abstract
            'acousticness_Min': 0.1, 'acousticness_Max': 0.8,
            'time_signature_Min': 3, 'time_signature_Max': 7, # Atypical signatures might fit
            'loudness_Min': -21, 'loudness_Max': -9,
            'speechiness_Min': 0.0, 'speechiness_Max': 0.4 # Could have disjointed speech/vocals
        })

    # --- OTHER DISTINCT EMOTIONS ---
    elif emotion == 'nostalgia':
        rule.update({
            'danceability_Min': 0.25, 'danceability_Max': 0.6,
            'energy_Min': 0.2, 'energy_Max': 0.55,
            'valence_Min': 0.4, 'valence_Max': 0.75,
            'tempo_Min': 70, 'tempo_Max': 115,
            'loudness_Min': -20, 'loudness_Max': -9,
            'acousticness_Min': 0.55, 'acousticness_Max': 1.0,
            'instrumentalness_Min': 0.05, 'instrumentalness_Max': 0.8,
            'liveness_Min': 0.0, 'liveness_Max': 0.25,
            'speechiness_Min': 0.0, 'speechiness_Max': 0.15
        })
    elif emotion == 'tradition':
        rule.update({
            'danceability_Min': 0.2, 'danceability_Max': 0.65,
            'energy_Min': 0.15, 'energy_Max': 0.55,
            'valence_Min': 0.35, 'valence_Max': 0.75,
            'tempo_Min': 60, 'tempo_Max': 130,
            'acousticness_Min': 0.7, 'acousticness_Max': 1.0,
            'instrumentalness_Min': 0.0, 'instrumentalness_Max': 0.7,
            'liveness_Min': 0.0, 'liveness_Max': 0.35,
            'speechiness_Min': 0.0, 'speechiness_Max': 0.25,
            'time_signature_Min': 3, 'time_signature_Max': 4,
            'loudness_Min': -22, 'loudness_Max': -10,
        })
    elif emotion == 'neutral':
        rule.update({
            'danceability_Min': 0.25, 'danceability_Max': 0.55,
            'energy_Min': 0.25, 'energy_Max': 0.55,
            'valence_Min': 0.35, 'valence_Max': 0.65, # Centered
            'tempo_Min': 75, 'tempo_Max': 115,
            'loudness_Min': -22, 'loudness_Max': -11,
            'speechiness_Min': 0.0, 'speechiness_Max': 0.1,
            'instrumentalness_Min': 0.4, 'instrumentalness_Max': 1.0, # Often background/instrumental
            'acousticness_Min': 0.25, 'acousticness_Max': 0.85,
            'liveness_Min': 0.05, 'liveness_Max': 0.25
        })
    elif emotion == 'disapproval' or emotion == 'envy' or emotion == 'deceit':
        rule.update({
            'danceability_Min': 0.2, 'danceability_Max': 0.6,
            'energy_Min': 0.35, 'energy_Max': 0.75,
            'valence_Min': 0.1, 'valence_Max': 0.4,
            'tempo_Min': 70, 'tempo_Max': 140,
            'mode_Min': 0, 'mode_Max': 0, # Often minor
            'loudness_Min': -17, 'loudness_Max': -7,
            'speechiness_Min': 0.03, 'speechiness_Max': 0.45,
        })
        if emotion == 'deceit':
            rule['tempo_Min'] = 65; rule['tempo_Max'] = 115;
            rule['loudness_Min'] = -20; rule['loudness_Max'] = -10;
            rule['energy_Min'] = 0.25; rule['energy_Max'] = 0.6;

    # If an emotion isn't specifically handled, it uses the broad default from create_default_rule()
    emotion_rules_list.append(rule)

In [28]:
lala = pd.DataFrame(emotion_rules_list, columns=columns)

In [29]:
lala

,Emotion,danceability_Min,danceability_Max,energy_Min,energy_Max,valence_Min,valence_Max,tempo_Min,tempo_Max,loudness_Min,...,mode_Min,mode_Max,speechiness_Min,speechiness_Max,acousticness_Min,acousticness_Max,instrumentalness_Min,instrumentalness_Max,liveness_Min,liveness_Max
0,suspicion,0.10,0.40,0.15,0.45,0.10,0.35,60,110,-22,...,0,0,0.00,0.33,0.20,0.80,0.40,1.00,0.00,1.00
1,disappointment,0.10,0.40,0.15,0.40,0.05,0.30,60,100,-22,...,0,0,0.00,0.33,0.40,0.95,0.00,1.00,0.00,1.00
2,hope,0.25,0.65,0.30,0.65,0.55,0.90,80,125,-17,...,0,1,0.00,0.33,0.30,0.90,0.00,1.00,0.00,1.00
3,curiosity,0.30,0.70,0.40,0.75,0.30,0.70,80,130,-18,...,0,1,0.00,0.33,0.10,0.70,0.40,1.00,0.00,1.00
4,amusement,0.65,0.90,0.60,0.85,0.70,1.00,110,140,-10,...,1,1,0.05,0.40,0.00,1.00,0.00,1.00,0.00,1.00
5,realization,0.20,0.60,0.25,0.65,0.25,0.70,60,120,-20,...,0,1,0.00,0.33,0.30,0.90,0.30,0.90,0.00,1.00
6,shame,0.15,0.45,0.20,0.45,0.10,0.30,65,105,-21,...,0,0,0.00,0.25,0.00,1.00,0.00,1.00,0.00,1.00
7,reassurance,0.20,0.60,0.25,0.60,0.65,0.95,70,120,-18,...,1,1,0.00,0.33,0.40,1.00,0.00,1.00,0.00,1.00
8,envy,0.20,0.60,0.35,0.75,0.10,0.40,70,140,-17,...,0,0,0.03,0.45,0.00,1.00,0.00,1.00,0.00,1.00
9,disgust,0.10,0.40,0.25,0.60,0.00,0.15,70,130,-19,...,0,0,0.00,0.30,0.00,1.00,0.00,1.00,0.00,1.00


In [30]:
lala.to_csv('EmotionsWithFeatures.csv', index=False)

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
tokenizer = AutoTokenizer.from_pretrained('mbert')
model = AutoModelForSequenceClassification.from_pretrained('mbert').to(device)

In [33]:
loaded_mlb = MultiLabelBinarizer()
loaded_mlb.fit(ax['emotion'])

MultiLabelBinarizer()

In [34]:
ax

,text,emotion
0,[RU] Эта игра причинила мне боль.,[sadness]
1,"[RU] Ты правильно поступаешь, если тебе все ра...",[neutral]
2,"[RU] Чувак, я обожаю Reddit.",[love]
3,"[RU] [ИМЯ] не было рядом с ними, он был рядом ...",[neutral]
4,"[RU] Верно? Учитывая, что это такой важный док...",[gratitude]
...,...,...
479720,"[KZ] ""Сен онымен на хабаре болсаң, менің өтіні...","[desire, caring, nervousness]"
479721,"[KZ] ""Оның ""танкасы бар"" болса да, заң алдында...","[approval, determination, anger]"
479722,"[KZ] ""Ол шындықты айтып тұр ма, или өтірік пе,...","[curiosity, nervousness, confusion]"
479723,"[KZ] Университетте оқып жүргенде, біз общягада...","[nostalgia, caring, joy]"


In [35]:
print(loaded_mlb.classes_)

['admiration' 'amusement' 'anger' 'annoyance' 'anticipation' 'anxiety'
 'approv' 'approval' 'caring' 'caution' 'concern' 'confidence' 'confusion'
 'contentment' 'creativity' 'curiosity' 'deceit' 'desire' 'determination'
 'disappointment' 'disapproval' 'disbelief' 'discomfort' 'disgust'
 'embarrassment' 'envy' 'excitement' 'fear' 'frustration' 'gratitude'
 'grief' 'hope' 'joy' 'love' 'nervousness' 'neutral' 'nostalgia'
 'optimism' 'panic' 'passion' 'pride' 'realization' 'reassurance' 'regret'
 'relief' 'remorse' 'sadness' 'satisfaction' 'shame' 'shock' 'surprise'
 'suspicion' 'tradition' 'trust' 'urgency']


In [36]:
import joblib
import os

model_directory = 'mbert'
mlb_filename = 'mlb.joblib'
mlb_path = os.path.join(model_directory, mlb_filename)

joblib.dump(loaded_mlb, mlb_path)

['mbert/mlb.joblib']

In [37]:
def clean_text(text):
    text = str(text)

    match = re.match(r"\[(KZ|RU|EN)\]", text)
    lang_tag = match.group(0) if match else ""

    text_wo_tag = text.replace(lang_tag, "") if lang_tag else text

    text_wo_tag = text_wo_tag.lower()
    text_wo_tag = re.sub(r"http\S+|www\S+|https\S+", '', text_wo_tag)
    text_wo_tag = re.sub(r"\s+", " ", text_wo_tag).strip()

    return f"{lang_tag} {text_wo_tag}".strip() if lang_tag else text_wo_tag

In [38]:
def predict_single_text_emotions(text_input, model_to_use, tokenizer_to_use, mlb_to_use, device_to_use, threshold=0.1):
    model_to_use.eval()

    cleaned_text = clean_text(text_input)
    print(f"Original input: '{text_input}'")
    print(f"Cleaned input: '{cleaned_text}'")

    inputs = tokenizer_to_use(
        cleaned_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128 
    )
    input_ids = inputs['input_ids'].to(device_to_use)
    attention_mask = inputs['attention_mask'].to(device_to_use)

    with torch.no_grad():
        outputs = model_to_use(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    probabilities = torch.sigmoid(logits).cpu().numpy()

    binary_predictions = (probabilities >= threshold).astype(int)

    if not hasattr(mlb_to_use, 'classes_') or not mlb_to_use.classes_.size:
        print("Error: MultiLabelBinarizer not fitted or has no classes.")
        return ["Error: MLB not properly loaded or fitted."]
    
    if binary_predictions.shape[1] != len(mlb_to_use.classes_):
         error_msg = f"Error: Prediction shape mismatch. Model predicted for {binary_predictions.shape[1]} labels, but MLB knows {len(mlb_to_use.classes_)} labels."
         print(error_msg)
         return [error_msg]

    predicted_emotion_tuples = mlb_to_use.inverse_transform(binary_predictions)
    
    predicted_emotions_list = list(predicted_emotion_tuples[0]) if predicted_emotion_tuples and predicted_emotion_tuples[0] else ["no specific emotion detected"]
    
    return predicted_emotions_list

In [42]:
text = "Мен қатты шаршадым, біраз демалғым келеді"

prediction = predict_single_text_emotions(text, model, tokenizer, loaded_mlb, device)
prediction

Original input: 'Мен қатты шаршадым, біраз демалғым келеді'
Cleaned input: 'мен қатты шаршадым, біраз демалғым келеді'


['annoyance', 'embarrassment', 'nervousness', 'sadness']

In [ ]:
print(f"Loaded model's configured number of labels: {model.config.num_labels}")

Loaded model's configured number of labels: 55


In [ ]:
features = [
    'danceability', 'energy', 'valence', 'tempo', 'loudness',
    'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness'
]

X = data[features].values  # shape: (160k, 10)
min_bounds = lala[[f"{f}_Min" for f in features]].values  # shape: (num_emotions, 10)
max_bounds = lala[[f"{f}_Max" for f in features]].values


In [ ]:
# Expand dimensions for broadcasting
X_expanded = X[:, np.newaxis, :]  # shape: (160k, 1, 10)
min_bounds = min_bounds[np.newaxis, :, :]  # shape: (1, num_emotions, 10)
max_bounds = max_bounds[np.newaxis, :, :]

# Calculate distances outside the [min, max] range
lower_diff = np.clip(min_bounds - X_expanded, a_min=0, a_max=None)
upper_diff = np.clip(X_expanded - max_bounds, a_min=0, a_max=None)

# Total distances
total_distance = lower_diff + upper_diff  # shape: (160k, num_emotions, 10)
sum_distance = total_distance.sum(axis=2)  # shape: (160k, num_emotions)

# Final scores (1 / (1 + distance))
scores = 1 / (1 + sum_distance)


In [ ]:
threshold = 0.5
mask = scores > threshold  # shape: (160k, num_emotions)

emotion_names = lala['Emotion'].values
matching_emotions_list = []

for i in range(X.shape[0]):
    matches = [(emotion_names[j], scores[i, j]) for j in range(len(emotion_names)) if mask[i, j]]
    matching_emotions_list.append(matches)

data['Matching_Emotions'] = matching_emotions_list


In [ ]:
data[['Matching_Emotions', 'name']]

,Matching_Emotions,name
0,"[(hope, 0.7729169887154119), (curiosity, 0.575...",Comedy
1,"[(realization, 0.7102300773289986), (reassuran...",Ghost - Acoustic
2,"[(realization, 0.6578947368421053), (reassuran...",To Begin Again
3,"[(concern, 1.0)]",Can't Help Falling In Love
4,"[(hope, 0.7230657989877078), (curiosity, 0.652...",Hold On
...,...,...
164678,"[(concern, 1.0), (frustration, 0.8665511265164...",Ryusei Rocket
164679,"[(concern, 0.7407407407407438)]",Colors Of The Wind
164680,"[(concern, 1.0), (frustration, 0.9302325581395...",The Revelation
164681,"[(concern, 1.0), (frustration, 0.7158196134574...",Koi no Dependence


In [ ]:
target_emotions = prediction

def match_predicted_emotions(emotion_list, target_emotions):
    return any(emo in target_emotions for emo, score in emotion_list)

filtered = data[data['Matching_Emotions'].apply(
    lambda ems: match_predicted_emotions(ems, target_emotions)
)]


In [ ]:
def avg_matching_score(ems, target_emotions):
    scores = [score for emo, score in ems if emo in target_emotions]
    return sum(scores) / len(scores) if scores else 0

filtered['AvgScore'] = filtered['Matching_Emotions'].apply(
    lambda ems: avg_matching_score(ems, target_emotions)
)

filtered = filtered.sort_values(by='AvgScore', ascending=False)


/tmp/ipykernel_293808/2436391641.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['AvgScore'] = filtered['Matching_Emotions'].apply(


In [ ]:
count = len(filtered[filtered['AvgScore'] == 1.0])
count

1923

In [ ]:
filtered

,track_id,artists,name,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,popularity,Matching_Emotions,AvgScore
104011,5PDSxvzJnXM0WnhVPBUvMS,Aitana;Morat,Más De Lo Que Aposté,205031,0.621,0.918,1,-3.010,0,0.0552,0.075300,0.000000,0.1300,0.8380,151.988,4,NaN,"[(concern, 1.0), (frustration, 0.6341154090044...",1.000000
197,1HqVCK9h4LiJ61aRrrKTAt,Freddie King,Stumble,216626,0.678,0.612,4,-9.764,1,0.0339,0.100000,0.316000,0.1640,0.8570,118.381,4,0.0,"[(hope, 0.8143322475570033), (curiosity, 0.805...",1.000000
104045,4CrvuZnTyzReLCcXRMETLK,Manuel Carrasco,Yo Quiero Vivir,243093,0.690,0.733,0,-6.194,1,0.1050,0.143000,0.000000,0.1470,0.7040,117.969,4,NaN,"[(hope, 0.7812500000000001), (amusement, 1.0),...",1.000000
143278,1fujTda6zX9tsYTc4Vvgcu,Eric Prydz,Woz Not Woz,427066,0.682,0.850,6,-10.345,1,0.1030,0.008130,0.859000,0.1980,0.7380,127.958,4,NaN,"[(curiosity, 0.8130940668525941), (amusement, ...",1.000000
112443,2ta23QFnRMP0Bi7n87xGcp,Hidra,Merhamet,140727,0.719,0.949,1,-4.882,0,0.0752,0.645000,0.000000,0.0921,0.9150,140.074,4,NaN,"[(concern, 1.0), (frustration, 0.5608524957936...",1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65876,4mfj6ay93WO3lvTZ3jXYIR,Anuradha Paudwal,Niah Shank Hoi Re Mana,1345998,0.582,0.572,10,-7.739,1,0.0353,0.606000,0.000000,0.0905,0.8120,156.966,3,NaN,"[(concern, 1.0), (shock, 0.8992805755395683), ...",0.500075
149058,1O04bfLogDlsfFHWd4eKQl,Bayside,They Looked Like Strong Hands,295840,0.270,0.646,2,-6.057,1,0.0473,0.002260,0.000094,0.5640,0.1370,146.966,3,NaN,"[(concern, 1.0), (creativity, 0.70889527461752...",0.500075
132803,2GPv6DEckvnKE7Fr75dX8w,Whitechapel,Father Of Lies,243786,0.283,0.991,6,-4.577,1,0.1710,0.000332,0.019500,0.8030,0.0582,111.012,4,NaN,"[(concern, 1.0), (pride, 0.5883045064125191), ...",0.500050
148510,03woVa7txdGcBbYcL4Eb1V,Nevermore,The Politics Of Ecstasy,476466,0.335,0.974,11,-5.666,1,0.0633,0.000005,0.377000,0.4790,0.1820,109.807,4,NaN,"[(concern, 1.0), (creativity, 0.74515927582053...",0.500000


In [ ]:
model.push_to_hub('yarlspace/mbert')
tokenizer.push_to_hub('yarlspace/mbert')

CommitInfo(commit_url='https://huggingface.co/yarlspace/mbert/commit/4642653f2149d033f384885bc4cc8c68dc1bd770', commit_message='Upload tokenizer', commit_description='', oid='4642653f2149d033f384885bc4cc8c68dc1bd770', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yarlspace/mbert', endpoint='https://huggingface.co', repo_type='model', repo_id='yarlspace/mbert'), pr_revision=None, pr_num=None)

In [1]:
from huggingface_hub import upload_file

local_path_to_mlb = 'mbert/mlb.joblib' 

upload_file(
    path_or_fileobj=local_path_to_mlb, 
    path_in_repo='mlb.joblib', 
    repo_id='yarlspace/mbert',
    repo_type='model'
)


/home/yeaz/Desktop/git/sentiment/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CommitInfo(commit_url='https://huggingface.co/yarlspace/mbert/commit/8b19eaedad30473b07b379ab48938adfe8a745ea', commit_message='Upload mlb.joblib with huggingface_hub', commit_description='', oid='8b19eaedad30473b07b379ab48938adfe8a745ea', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yarlspace/mbert', endpoint='https://huggingface.co', repo_type='model', repo_id='yarlspace/mbert'), pr_revision=None, pr_num=None)